In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import glob
import urllib
import cv2
import re
%matplotlib inline

In [ ]:
path = r'../data/carmax'
all_files = glob.glob(path + "/*.xlsx")
li = []
for filename in all_files:
    df = pd.read_excel(filename, index_col=None, header=0)
    if 'coupe' in filename:
        df['type'] = 'coupe'
    elif 'crossover' in filename:
        df['type'] = 'crossover'
    elif 'sedan' in filename:
        df['type'] = 'sedan'
    elif 'suv' in filename:
        df['type'] = 'suv'
    elif 'truck' in filename:
        df['type'] = 'truck'
    li.append(df)

df = pd.concat(li, axis=0, ignore_index=True)

In [ ]:
df.info()

In [ ]:
df.head()

In [ ]:
df = df.drop(np.array(df[df.src == 'x'].index))
df = df.drop_duplicates()

In [ ]:
df.info()

In [ ]:
df["price"] = df["price"].apply(lambda x: int(re.sub("[^0-9]", "", x)))

In [ ]:
def miles_to_int(x):
    if x == 'x':
        x = '0'
    x = x.replace('K', '000')
    return int(re.sub("[^0-9]", "", x))

In [ ]:
df["mileage"] = df["mileage"].apply(lambda x: miles_to_int(x))

In [ ]:
df["year"] = df["model"].apply(lambda x: int(x.split()[0]))

In [ ]:
def get_make(x):
    make_model = x.split('\n')
    make = make_model[0].split(' ', 1)[1]
    return make

def get_trim(x):
    make_model = x.split('\n')
    return make_model[1]

In [ ]:
df["make"] = df["model"].apply(lambda x: get_make(x))

In [ ]:
df["trim"] = df["model"].apply(lambda x: get_trim(x))

In [ ]:
df = df.reset_index().drop('index', axis=1)

In [ ]:
df.head()

In [ ]:
df.iloc[641]['src']

In [ ]:
from urllib.error import HTTPError, URLError
import socket

In [ ]:
urls = df.iloc[:]['src']

In [ ]:
urls[10971:]

In [ ]:
for i in range(10971, len(urls)):
    print(urls[start])

In [ ]:
import os

In [ ]:
start_i = 0
for i in range(10984, len(urls)):
    cont = False
    count = 0
    while (not cont) and count < 10:
        try:
            img_file_path = os.path.join("../images/carmax/"+ str(start_i + i) + ".jpg")
            if i > 10971:
                print(img_file_path)
                urllib.request.urlretrieve(urls[i], img_file_path)
                print(urls[i])
                cont = True
        except (HTTPError, URLError) as error:
            print("Error occurred at " + str(i))
            count += 1
        

In [ ]:
def is_original(image):
    a = cv2.imread('../images/carmax/894.jpg')
    b = cv2.imread(image)
    try:
        difference = cv2.subtract(a, b)    
        result = not np.any(difference)
        if result is True:
            return 'sample'
        else:
            return 'original'
    except:
        try:
            c = cv2.imread('../images/carmax/7751.jpg')
            difference = cv2.subtract(c, b)    
            result = not np.any(difference)
            if result is True:
                return 'sample'
            else:
                return 'original'
        except:    
            return 'original'

In [ ]:
path = r'../images/carmax'
all_files = glob.glob(path + "/*.jpg")
originals = []
for i in range(len(all_files)):
    originals.append(is_original('../images/carmax/' + str(i) + '.jpg'))

In [ ]:
df['image-type'] = originals

In [ ]:
df[df['image-type'] == 'sample'].info()

In [ ]:
from PIL import Image, ImageOps

In [ ]:
desired_size = 128
im_pth = "../images/carmax/0.jpg"
im = Image.open(im_pth)
old_size = im.size
ratio = float(desired_size)/max(old_size)
new_size = tuple([int(x*ratio) for x in old_size])
im = im.resize(new_size, Image.ANTIALIAS)
new_im = Image.new("RGB", (desired_size, desired_size))
new_im.paste(im, ((desired_size-new_size[0])//2,
                    (desired_size-new_size[1])//2))
new_im.save()

The following is to extract objects from the image. Because this will take too long to do for 50000 images, we will skip this step

In [ ]:
# from imageai.Detection import ObjectDetection
# import os

In [ ]:
# execution_path = os.getcwd()

# detector = ObjectDetection()
# detector.setModelTypeAsRetinaNet()
# detector.setModelPath( os.path.join(execution_path , "resnet50_coco_best_v2.0.1.h5"))
# detector.loadModel()
# detections = detector.detectObjectsFromImage(input_image=os.path.join(execution_path , "../images/carmax/4.jpg"), output_image_path=os.path.join(execution_path , "imagenew.jpg"))

# for eachObject in detections:
#     print(eachObject["name"] , " : " , eachObject["percentage_probability"] )

In [ ]:
df['model'] = 'undefined'

In [ ]:
df.head()

In [ ]:
df.info()

In [ ]:
df.to_csv('../data/carmax_final_data.csv')